<a href="https://colab.research.google.com/github/AditMeh/cuda-autograd/blob/main/testing_cu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [29]:
%%cu
#include <cstdio>
#include <iostream>
#include <stdio.h>

__global__ void matmulkernel(float*vec1, float* vec2, float *result){
    int i = threadIdx.x;
    result[i] = vec1[i] + vec2[i]; // vec addition
}

void setrand(float *arr, int len){
    for (int i =0; i< len; i++) {
        // arr[i] = float((double)rand()/(double)(RAND_MAX/9));
        arr[i] = float(rand() % 10);

    }
}

void printvec(float *arr, int len){
    printf("[");
    for (int i =0; i < len; i++) {
        printf("%f ", arr[i]);
    }
    printf("]");

    printf("\n\n");
}
int main() {

    float* vec1d; float* vec2d; float* resultd; // the d means device
    int len = 5;


    // Make the first and second vector and put it on the GPU
    float vec1[len];
    setrand(vec1, len);
    cudaMalloc((void **) &vec1d, len * sizeof(float));
    cudaMemcpy((void*)vec1d, vec1, len * sizeof(float), cudaMemcpyHostToDevice);

    float vec2[len];
    setrand(vec2, len);
    cudaMalloc((void **) &vec2d, len * sizeof(float));
    cudaMemcpy(vec2d, vec2, len * sizeof(float), cudaMemcpyHostToDevice);

    printvec(vec1, len);
    printvec(vec2, len);


    float result[len];
    cudaMalloc((void **) &resultd, len * sizeof(float));

    dim3 dimBlock(len);
    dim3 dimGrid(1);

    matmulkernel<<<dimGrid, dimBlock>>>(vec1d, vec2d, resultd);
    cudaMemcpy(result, resultd, len * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(vec1d); cudaFree(vec2d); cudaFree(resultd);

    printvec(result, len);
    cudaDeviceSynchronize();
    return 0;
}


[3.000000 6.000000 7.000000 5.000000 3.000000 ]

[5.000000 6.000000 2.000000 9.000000 1.000000 ]

[-456038850857154633913782602039296.000000 0.000000 -0.000001 0.000000 -0.000000 ]


